BERT: encoding, predicts words based on surrounding context.

GPT: decoding, uses context from left to right to predict the next word in a sentence

## **Checkpoint**
https://drive.google.com/drive/folders/1ii_lPm2-1CfIhQM8RVzLgTHMxXDKgnk4?usp=sharing

#**Low-resource Machine Translation using mBART50**

In [ ]:
# !pip install -q transformers sentencepiece datasets accelerate evaluate sacrebleu
# # sentencepiece to encode,

In [ ]:
# Imports
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, AutoModelForSeq2SeqLM
import torch
from google.colab import drive

In [ ]:
## Config
# Định nghĩa lớp cấu hình cơ bản
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

# Định nghĩa lớp cấu hình cho bài toán dịch thuật
class NMTConfig(BaseConfig):
    # Data
    src_lang = 'en'  # Ngôn ngữ nguồn
    tgt_lang = 'vi'  # Ngôn ngữ đích
    max_len = 75  # Độ dài tối đa của câu
    add_special_tokens = True  # Thêm các token đặc biệt

    # Model
    model_name = "facebook/mbart-large-50-many-to-many-mmt"  # Tên mô hình

    # Training
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # Thiết bị (GPU hoặc CPU)
    learning_rate = 5e-5  # Tốc độ học
    train_batch_size = 16  # Kích thước batch khi huấn luyện
    eval_batch_size = 16  # Kích thước batch khi đánh giá
    num_train_epochs = 2  # Số epoch huấn luyện
    save_total_limit = 1  # Số lượng checkpoint lưu trữ
    ckpt_dir = f'./mbart50-{src_lang}-{tgt_lang}'  # Đường dẫn lưu trữ checkpoint
    eval_steps = 1000  # Số bước mỗi lần đánh giá

    # Inference
    beam_size = 5  # Kích thước beam search

cfg = NMTConfig()

In [ ]:

# Định nghĩa tên model: Bài này mình xài chung model này làm model và tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"

# Tokenizer and add to cfg.tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained(model_name, src_lang="en_XX", tgt_lang="vi_VN")
cfg.tokenizer = MBart50TokenizerFast.from_pretrained(cfg.model_name)

# Định nghĩa tên mô hình và tải tokenizer từ hugging
MBart50_model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_name)


# Tải mô hình từ checkpoint
drive.mount('/content/drive/') # Mount Google Drive

checkpoint_path = "/content/drive/MyDrive/colab/model/mbart50-en-vi/checkpoint-16000"
mbart50_envi_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)


# checkpoint_path = "/content/drive/MyDrive/colab/model/mbart50-en-vi-backtranslation/checkpoint-9000"
# mbart50_envi_backtranslation_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)

In [ ]:
!pip install --upgrade transformers


In [ ]:
def inference(
    text,
    tokenizer,
    model,
    device="cpu",
    max_length=75,
    beam_size=5
    ):
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
        )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        early_stopping=True,
        num_beams=beam_size,
        length_penalty=2.0
    )

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str

In [ ]:
sentence = 'i go to school'
inference(sentence, cfg.tokenizer, model)

# Tính toán BLEU

In [ ]:
import evaluate
import torch

# Define sentences to test and their references
sentences = [
    'i go to school',
    'she loves programming',
    'he is reading a book'
]
references = [
    ['tôi đi học'],
    ['cô ấy yêu lập trình'],
    ['anh ấy đang đọc sách']
]

# Load BLEU metric
bleu = evaluate.load("sacrebleu")

# Function to evaluate a model and print BLEU score
def evaluate_model(model, model_name, device="cpu"):
    model.to(device)
    predictions = [inference(sentence, cfg.tokenizer, model, device=device)[0] for sentence in sentences]
    results = bleu.compute(predictions=predictions, references=references)
    print(f"BLEU score for {model_name}: {results['score']}")
    print(f"Predictions: {predictions}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Evaluate mbart50_envi_model
evaluate_model(mbart50_envi_model, "mbart50_envi_model", device)

# Evaluate MBart50_model
evaluate_model(MBart50_model, "MBart50_model", device)

# Evaluate mbart50_envi_backtranslation_model
evaluate_model(mbart50_envi_backtranslation_model, "mbart50_envi_backtranslation_model", device)


In [ ]:
# Đánh giá trên tập test.en, test.vi

In [ ]:
# Cài đặt thư viện Kaggle
!pip install kaggle

# Tạo thư mục để lưu trữ kaggle.json
!mkdir -p ~/.kaggle

# Di chuyển tệp kaggle.json vào thư mục .kaggle và cấp quyền
!mv /content/drive/MyDrive/colab/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Kiểm tra sự tồn tại của tệp kaggle.json
!ls ~/.kaggle/

# Kiểm tra nội dung của tệp kaggle.json (chỉ kiểm tra trong môi trường an toàn)
!cat ~/.kaggle/kaggle.json

# Kiểm tra quyền truy cập của tệp kaggle.json
!ls -l ~/.kaggle/



In [ ]:
# Tải dữ liệu từ Kaggle
!kaggle datasets download -d cngonngc/phomt-datasetvinai

# Giải nén tệp dữ liệu đã tải về
import zipfile

with zipfile.ZipFile('phomt-datasetvinai.zip', 'r') as zip_ref:
    zip_ref.extractall('phomt-datasetvinai')

# Kiểm tra các tệp đã được giải nén
!ls phomt-datasetvinai


In [ ]:
import os

# Định nghĩa đường dẫn tới các tệp test
test_en_path = 'phomt-datasetvinai/test.en'
test_vi_path = 'phomt-datasetvinai/test.vi'

# Đọc dữ liệu từ các tệp
with open(test_en_path, 'r') as f:
    test_sentences_en = f.readlines()

with open(test_vi_path, 'r') as f:
    test_sentences_vi = f.readlines()

# Lựa chọn một mẫu nhỏ để kiểm tra
sample_size = 10  # Chọn 10 câu để kiểm tra
sample_en = test_sentences_en[:sample_size]
sample_vi = test_sentences_vi[:sample_size]

# Generate translations
predictions = [inference(sentence.strip(), cfg.tokenizer, model)[0] for sentence in sample_en]

# Load BLEU metric
bleu = evaluate.load("sacrebleu")

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=[[ref.strip()] for ref in sample_vi])

print(f"BLEU score: {results['score']}")
print(f"Predictions: {predictions[:5]}")
print(f"References: {sample_vi[:5]}")
